In [ ]:
import json
import numpy as np
from openai import OpenAI
from order_by.sorting import *
from order_by.utils import *

with open(".open_ai_api_key", "r") as f:
    api_key = f.read().strip()

client = OpenAI(api_key=api_key,)

# Sample and save data

In [ ]:
# import datasets

# data = datasets.load_dataset("ms_marco", "v1.1", split="train")
# sampled_data = data.shuffle(seed=42).select(range(100))

# data_list = [dict(item) for item in sampled_data]
# with open("msmarco_v1.1_sampled_100.json", "w") as f:
#     json.dump(data_list, f, indent=2)

# print("Saved to 'msmarco_v1.1_sampled_100.json'")

# Load Experiment Data

In [ ]:
with open("msmarco_v1.1_sampled_100.json", "r") as f:
    data = json.load(f)

# Prompts Template

In [ ]:
pointwise_prompt_template = """You are given a question and a passage. Evaluate how well the passage answers the question by assigning a score from 0 to 5.\nOutput only a float score.\nQuestion: {question}\nPassage: {passage}\nScore:"""

external_pointwise_prompt_template = """You are given a question and a list of passages. Evaluate how well each passage answers the question. For each passage, assign a float score from 0 to 5.\nOutput a JSON list of float scores in the same order as the input passages.\nQuestion: {question}\nPassages:\n{numbered_passages}\nScore:"""

pairwise_comparison_prompt_template = """You are given a question and two passages. Determine which passage answers the question better.\nQuestion: {question}\nPassage A: {passage_a}\nPassage B: {passage_b}\nOutput only one word: 'A' if Passage A is better, 'B' if Passage B is better, or 'Equal' if both are equally good."""

external_comparison_prompt_template = """You are given a question and a list of passages. Rank the passages based on how well they answer the question, from best to worst.\nQuestion: {question}\nPassages:\n{numbered_passages}\nOutput a JSON list of passage numbers in ranked order (best to worst)."""

In [ ]:
def create_numbered_passages(passages):
    return "\n".join([f"{i+1}. {p}" for i, p in enumerate(passages)])

def precision(selected_ground_truth, output):
    """ Evaluate the precision of the ranked result.
    precision@k = (#relevant output items in top k / k) where k is the number of relevant items in selected_ground_truth.
    """
    selected_ground_truth = np.asarray(selected_ground_truth)
    k = np.sum(selected_ground_truth, axis=1)
    idxs = np.asarray(output) - 1
    rows = np.arange(idxs.shape[0])[:, None]
    relevance = selected_ground_truth[rows, idxs]
    rank_positions = np.arange(idxs.shape[1])
    mask = rank_positions < k[:, None]
    return np.sum(mask * relevance, axis=1) / k

In [ ]:
for i, s in enumerate(data):
    query = s['query']
    passage = s['passages']['passage_text']
    selected = s['passages']['is_selected']
    if sum(selected) == 0:
        print(f"skip bad experiment {i}")
        print(selected)
        continue
    
    # print(pointwise_prompt_template.format(question=query, passage=passage[0]))
    # print(external_pointwise_prompt_template.format(question=query, numbered_passages=create_numbered_passages(passage)))
    # print(pairwise_comparison_prompt_template.format(question=query, passage_a=passage[0], passage_b=passage[1]))
    # print(external_comparison_prompt_template.format(question=query, numbered_passages=create_numbered_passages(passage)))